In [1]:
import os
os.chdir('../')
%pwd

'h:\\Personal Projects\\Indian-Housing-Prices-Backend'

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str

In [3]:
from houseProject.constants import *
from houseProject.utils.common import read_yaml, create_directories, save_json

In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.CatBoostRegressor
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name
           
        )

        return model_evaluation_config

In [5]:
import os
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, median_absolute_error, explained_variance_score
from urllib.parse import urlparse
import numpy as np
import joblib

In [6]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def evaluate_model(self, actual, pred):
        mae = mean_absolute_error(actual, pred)
        mse = mean_squared_error(actual, pred)
        rmse = np.sqrt(mse)
        r2_square = r2_score(actual, pred)
        med_ae = median_absolute_error(actual, pred)
        evs = explained_variance_score(actual, pred)
        
        return mae, mse, rmse, r2_square, med_ae, evs

    


    def save_results(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]
        
        predicted_qualities = model.predict(test_x)

        (mae, mse, rmse, r2, med_ae, evs) = self.evaluate_model(test_y, predicted_qualities)
        
        # Saving metrics as local
        scores = {'MAE': mae, 'MSE': mse, 'RMSE': rmse, 'R2 Score': r2, 'Median AE': med_ae, 'Explained Variance Score': evs}
        save_json(path=Path(self.config.metric_file_name), data=scores)

In [7]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.save_results()
except Exception as e:
    raise e

[2025-05-19 20:56:47,246: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-19 20:56:47,246: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-19 20:56:47,246: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-05-19 20:56:47,246: INFO: common: created directory at: artifacts]
[2025-05-19 20:56:47,261: INFO: common: created directory at: artifacts/model_evaluation]
[2025-05-19 20:56:49,294: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]
